## Installation of Streamlit UI

In [ ]:
from IPython.display import IFrame


In [ ]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path


In [ ]:

region = boto3.Session().region_name
client = boto3.Session().client("sagemaker")
domainId=client.list_domains()["Domains"][0].get("DomainId")
sess= sagemaker.session.Session()
region = sess._region_name
account_id = sess.account_id()

vpc_id = client.describe_domain(
    DomainId=client.list_domains()["Domains"][0].get("DomainId")
).get("VpcId")

url = f"https://{domainId}.studio.{region}.sagemaker.aws/jupyter/default/proxy/8501/"
print(url )

# Generative AI Prompt example with Falcon

## Use case : Data extraction

<p>Let's say you want to extract structured data from sport news. You can use <b>few shot prompting</b> to extract relevant information such as names, dates, numbers, ect </p> 

In the example below,I've collected news about soccer transfer information and the goal is to extract a structured JSON document with :
- Sport
- Person involve in the transfer
- Club
- Money involve 

- **Parameters** : max_new_tokens = 30 <p>

```
Text:
West Ham have just communicated to Arsenal that they’re accepting £100m plus £5m add-ons fee for Declan Rice. The two clubs remain in talks over deal structure & payment terms — as West Ham want £100m to be paid within 18 months. Final discussions and then… done deal.

JSON 
{sport : foot, who:Declan Rice, club : West Ham ,money:  £100m }

Text:
BREAKING: Manchester United agree £60m package deal for Mason Mount with Chelsea — it’s done, here we go! 
 #MUFC Personal terms agreed weeks ago and face to face talks between clubs made it clear: the agreement is done. Mount becomes Utd player — they NEVER left the race.


JSON 
{sport : foot, who: Mason Mount, club:Manchester United,  money:   £60m }

Text:
James Maddison has completed first part of medical tests as new Tottenham player, as expected. 
#THFC £40m deal to be announced soon — here we go confirmed.

JSON 
{sport : foot, who: James Maddison : club: Tottenham , money:   £60m }

Text: 
Manchester United had new direct talks to be informed on André Onana deal in the last 24 hours. He’s highly rated by Erik ten Hag. 🔴🇨🇲 #MUFC
Man Utd, still waiting to clarify David de Gea situation before deciding on official proposal.

Inter want at least €50/55m.

JSON
{sport : foot, who: David De Gea/André Onana  , club : Manchester United , money: €50/55}

Text:
Medical scheduled for Harry Winks to join Leicester City on permanent deal from Tottenham for £10m fee. 🔵🦊 #LCFC. Separated deal from Maddison but one more set to be completed.

JSON
```

### Code generation

##### Example 1
- **Parameters** : max_new_tokens = 200 <p>

```
Building a website can be done in 10 simple steps
```


##### Example 2
- **Parameters** : max_new_tokens = 110 <p>
```
Building a website can be done in 10 simple steps  
```

#### Sentiment Analysis
- **Parameters** : max_new_tokens = 8 <p>
```
I hate it when my phone battery dies."
Sentiment: Negative
###
Tweet: "My day has been :+1:"
Sentiment: Positive
###
Tweet: "This is the link to the article"
Sentiment: Neutral
###
Tweet: "This new music video was incredibile"
Sentiment:
```

#### Text Generation
- **Parameters** : max_new_tokens = 400 <p>
```
What is the recipe for a delicious lemon cheesecake?
```
                

#### Summarize
- **Parameters** : max_new_tokens = 200 <p>
```
Starting today, the state-of-the-art Falcon 40B foundation model from Technology
Innovation Institute (TII) is available on Amazon SageMaker JumpStart, SageMaker's machine learning (ML) hub
that offers pre-trained models, built-in algorithms, and pre-built solution templates to help you quickly get
started with ML. You can deploy and use this Falcon LLM with a few clicks in SageMaker Studio or
programmatically through the SageMaker Python SDK.
Falcon 40B is a 40-billion-parameter large language model (LLM) available under the Apache 2.0 license that
ranked #1 in Hugging Face Open LLM leaderboard, which tracks, ranks, and evaluates LLMs across multiple
benchmarks to identify top performing models. Since its release in May 2023, Falcon 40B has demonstrated
exceptional performance without specialized fine-tuning. To make it easier for customers to access this
state-of-the-art model, AWS has made Falcon 40B available to customers via Amazon SageMaker JumpStart.
Now customers can quickly and easily deploy their own Falcon 40B model and customize it to fit their specific
needs for applications such as translation, question answering, and summarizing information.
Falcon 40B are generally available today through Amazon SageMaker JumpStart in US East (Ohio),
US East (N. Virginia), US West (Oregon), Asia Pacific (Tokyo), Asia Pacific (Seoul), Asia Pacific (Mumbai),
Europe (London), Europe (Frankfurt), Europe (Ireland), and Canada (Central),
with availability in additional AWS Regions coming soon. To learn how to use this new feature,
please see SageMaker JumpStart documentation, the Introduction to SageMaker JumpStart –
Text Generation with Falcon LLMs example notebook, and the blog Technology Innovation Institute trainsthe
state-of-the-art Falcon LLM 40B foundation model on Amazon SageMaker. Summarize the article above:
```
      

### Supported parameters

***
Some of the supported parameters while performing inference are the following:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. 

For more parameters and information on HF LLM DLC, please see [this article](https://huggingface.co/blog/sagemaker-huggingface-llm#4-run-inference-and-chat-with-our-model).
***